In [78]:
import os
import sys
import pandas as pd

# 添加项目根目录到 Python 路径
project_root = '/date0/crwu/zjts_foodname'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from packages.file_deal import read_dataset

In [79]:
csv_data_3 = r"/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/food_name_database_V100R25C20.csv"

df = read_dataset(csv_data_3)
for row in df.iterrows():
    index = row[0]
    for key in ['食用频率', '熟悉程度']:
        if row[1][key].isdigit():
            df.loc[index,key] = int(row[1][key])
        else:
            # 不适用的改成-1方便后续对比
            df.loc[index,key] = -1

2025-10-14 23:03:43 - file_deal - INFO - [file_deal.py:151] - Loading dataset from CSV: /date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/food_name_database_V100R25C20.csv ...


In [80]:
import pandas as pd
import numpy as np
from collections import defaultdict


# 辅助函数：获取可用的样本索引
def get_valid_indices(df):
    invalid_set = set()
    for row in df.iterrows():
        index = row[0]
        if False or \
            '不适用' == row[1]['第一级别'] or \
                '(非菜品)' == row[1]['第一级别'] or \
                '(不适用)' == row[1]['第一级别']:                    
            invalid_set.add(index)
            continue        
        # print(row[1])
        # for _, val in row[1].items():
        #     if (val.startswith('(') and val.endswith(')')) or val == '不适用':                  
        #         invalid_set.add(index)
        #         print(row[1])
        #         break
    return set(df.index) - invalid_set
            
def iterative_conditional_sampling_fuzzy_optimized(df, sample_size, conditions, sep='/', random_state=None):
    """
    优化版：支持模糊匹配的迭代式条件满足抽样方法
    
    参数:
    df: 原始DataFrame
    sample_size: 需要抽取的总样本数
    conditions: 字典，包含各列的最小数量要求
    sep: 组合值中的分隔符，默认为'/'
    random_state: 随机种子，保证可重复性
    """
    if random_state is not None:
        np.random.seed(random_state)
    
    # 预处理：为每列创建值到索引的映射（支持模糊匹配）
    value_to_indices = defaultdict(lambda: defaultdict(set))
    
    for col in conditions.keys():
        for idx, value in df[col].items():
            cell_value = str(value)
            # 如果单元格值包含分隔符，则拆分为多个值
            if sep in cell_value:
                values = [v.strip() for v in cell_value.split(sep)]
                for v in values:
                    value_to_indices[col][v].add(idx)
            else:
                # 精确匹配
                value_to_indices[col][cell_value].add(idx)
    
    # 初始化已选样本和条件跟踪器
    selected_indices = []
    condition_tracker = defaultdict(lambda: defaultdict(int))
    
    # 将条件转换为更易处理的形式
    condition_list = []
    for col, value_requirements in conditions.items():
        for value, min_count in value_requirements.items():
            condition_list.append((col, value, min_count))
    
    # 辅助函数：更新条件跟踪器（支持模糊匹配）
    def update_condition_tracker(sample):
        for col in conditions.keys():
            cell_value = str(sample[col])
            # 如果单元格值包含分隔符，则拆分为多个值
            if sep in cell_value:
                values = [v.strip() for v in cell_value.split(sep)]
                for value in values:
                    condition_tracker[col][value] += 1
            else:
                condition_tracker[col][cell_value] += 1
    
    # 迭代抽样过程
    iteration = 0
    max_iterations = sample_size * 10  # 防止无限循环
    all_valida_index_set = get_valid_indices(df)
    while len(selected_indices) < sample_size and iteration < max_iterations:
        iteration += 1
        
        # 检查哪些条件尚未满足
        unmet_conditions = []
        for col, value, min_count in condition_list:
            if condition_tracker[col][value] < min_count:
                unmet_conditions.append((col, value, min_count))
        
        # 如果没有未满足的条件，则随机抽取一个样本
        if not unmet_conditions:
            remaining_indices = set(df.index) - set(selected_indices)
            if not remaining_indices:
                break  # 没有更多样本可用
            new_index = np.random.choice(list(remaining_indices))
            selected_indices.append(new_index)
            
            # 更新条件跟踪器
            new_sample = df.loc[new_index]
            update_condition_tracker(new_sample)
            continue
        
        # 随机选择一个未满足的条件
        col, value, min_count = unmet_conditions[np.random.randint(0, len(unmet_conditions))]
        
        # 使用预处理的结果查找满足该条件的可用样本
        available_indices = value_to_indices[col].get(value, set()) - set(selected_indices)
        
        if available_indices:
            # 随机选择一个满足条件的样本
            new_index = np.random.choice(list(available_indices))
            selected_indices.append(new_index)
            
            # 更新条件跟踪器
            new_sample = df.loc[new_index]
            update_condition_tracker(new_sample)
    
    # 如果还有剩余名额，随机补齐
    if len(selected_indices) < sample_size:
        remaining_indices = set(df.index) - set(selected_indices)
        additional_needed = sample_size - len(selected_indices)
        
        if len(remaining_indices) >= additional_needed:
            additional_selected = np.random.choice(
                list(remaining_indices), additional_needed, replace=False
            ).tolist()
            selected_indices.extend(additional_selected)
            
            # 更新条件跟踪器
            for idx in additional_selected:
                update_condition_tracker(df.loc[idx])
        else:
            print(f"警告: 只能抽取 {len(selected_indices) + len(remaining_indices)} 个样本")
            selected_indices.extend(remaining_indices)
            
            # 更新条件跟踪器
            for idx in remaining_indices:
                update_condition_tracker(df.loc[idx])
    
    # 返回抽样结果
    result_df = df.loc[selected_indices]
    
    # 验证结果
    res = validate_sample_fuzzy(result_df, conditions, sep)
    
    return result_df, res

def validate_sample_fuzzy(sampled_df, conditions, sep='/'):
    """验证抽样结果是否满足条件（支持模糊匹配）"""
    print("抽样结果验证:")
    print(f"总样本数: {len(sampled_df)}")
    
    # 计算每个条件的实际计数
    actual_counts = defaultdict(lambda: defaultdict(int))
    for col in conditions.keys():
        for idx, row in sampled_df.iterrows():
            cell_value = str(row[col])
            if sep in cell_value:
                values = [v.strip() for v in cell_value.split(sep)]
                for value in values:
                    actual_counts[col][value] += 1
            else:
                actual_counts[col][cell_value] += 1
    
    # 验证条件是否满足
    all_conditions_met = True
    for col, value_requirements in conditions.items():
        print(f"\n列 {col}:")
        for value, min_count in value_requirements.items():
            actual_count = actual_counts[col][value]
            status = "✓" if actual_count >= min_count else "✗"
            if actual_count < min_count:
                all_conditions_met = False
            print(f"  {value}: 要求 {min_count}, 实际 {actual_count} {status}")
    
    if all_conditions_met:
        print("\n✓ 所有条件均已满足!")
    else:
        print("\n✗ 部分条件未满足!")
    
    return all_conditions_met

# 使用示例
if __name__ == "__main__":
    pass

# 淡水鱼和海水鱼是手动删掉合并的，切记切记

In [81]:
import random
csv_root = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/'
for score in range(1,6):
    res = df[df['食用频率'] == (score)]
    print(f'食用频率为{score}的有{len(res)}个')
    print(random.sample(res.iloc[:, 0].to_list(), 10))

print('*' * 50)
for score in range(1,6):
    res = df[df['熟悉程度'] == (score)]
    print(f'熟悉程度为{score}的有{len(res)}个')
    print(random.sample(res.iloc[:, 0].to_list(), 10))
print('*' * 50)
print(set(df['食用频率']))

食用频率为1的有4316个
['鹅掌汤', '紫椰菜芋泥酥', '油旋', '法式红酒煎鹅肝', '婴儿辅食', '棒棒糖面包', '炸灌肠', '仙游海蛎饼', '跳水兔', '白兰地椰香馅香兰冰皮']
食用频率为2的有7576个
['糍糕', '南瓜塔', '玉米粑粑', '芝麻汤圆', '糟辣鱼', '紫薯山药糕', '芥菜粥', '和风玉子烧', '白桃罐头', '香蕉果酱']
食用频率为3的有7921个
['骨头粥', '紫薯银耳莲子羹', '素炒饼', '串串香', '淮南牛肉汤', '葱炒花蛤', '花生猪骨汤', '盐水鸡', '红烧碟鱼', '三鲜烧麦']
食用频率为4的有5820个
['煎饺子', '椰树椰汁', '油菜土豆片', '胡萝卜煎蛋', '白灼豆腐', '鸡翅炖土豆', '土豆豆角焖面', '胡萝卜素包子', '咸花生', '豆腐炒鸡蛋']
食用频率为5的有1129个
['菜汤', '西红柿豆腐汤', '菠菜炒鸡蛋', '蛋汤', '彩椒炒肉', '炝白菜', '排骨胡萝卜玉米汤', '白菜炖豆腐粉条', '猪肉酸菜馅包子', '鸡丁炒饭']
**************************************************
熟悉程度为1的有1765个
['菠菜豆沙饼', '爆球迎春', '炒芒果', '杠子头火烧', '阿胶鸡蛋羹', '法式焦糖杏仁酥', '鱿鱼米肠', '国王面包', '孢子甘蓝沙拉', '蓝波芝士']
熟悉程度为2的有3981个
['海凉粉', '暴腌武昌鱼', '四川凉糕', '海鲜墨鱼汁意大利面', '炒薄壳', '法棍迷你披萨', '葱烧蹄筋', '明太鱼籽', '紫薯小小酥', '曲靖蒸饵丝']
熟悉程度为3的有6076个
['剁椒芋艿', '日式泡菜', '蝴蝶面', '豆腐肉末酿香菇', '蒸米糕', '红枣山药羹', '乳鸽汤', '鲍鱼炖鸡', '蛋黄米粉球', '血橙']
熟悉程度为4的有9287个
['军舰寿司', '高汤娃娃菜', '塔皮', '炝拌莴笋丝', '烤鳗鱼', '山药炒木耳', '白菜蘑菇', '辣白菜炒肉', '干煸菜花', '干煸土豆片']
熟悉程度为5的有5655个
['意大利肉酱面', '杏鮑菇', '猪肉芹菜水饺', '蒸鱼', '泡菜汤', '茶

In [82]:
csv_data_test = r"/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_test_V100R25C20/food_name_database_500_test_V100R25C20.csv"

df_test = read_dataset(csv_data_test)
for row in df_test.iterrows():
    index = row[0]
    for key in ['食用频率', '熟悉程度']:
        if row[1][key].isdigit():
            df_test.loc[index,key] = int(row[1][key])
        else:
            # 不适用的改成-1方便后续对比
            df_test.loc[index,key] = -1
            
for score in range(1,6):
    res = df_test[df_test['食用频率'] == (score)]
    print(f'食用频率为{score}的有{len(res)}个')
    print(random.sample(res.iloc[:, 0].to_list(), min(10,len(res))))

print('*' * 50)
for score in range(1,6):
    res = df_test[df_test['熟悉程度'] == (score)]
    print(f'熟悉程度为{score}的有{len(res)}个')
    print(random.sample(res.iloc[:, 0].to_list(), min(10,len(res))))
print('*' * 50)
print(set(df_test['食用频率']))

2025-10-14 23:03:46 - file_deal - INFO - [file_deal.py:151] - Loading dataset from CSV: /date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_test_V100R25C20/food_name_database_500_test_V100R25C20.csv ...
食用频率为1的有42个
['九转大肠', '灰豆子', '龙虎斗', '土笋冻', '葱烧海参', '蟠龙菜', '炖甲鱼', '叫花鸡', '拆烩鲢鱼头', '松鼠鳜鱼']
食用频率为2的有95个
['鱼头泡饼', '擂椒皮蛋', '糍粑', '避风塘炒蟹', '蚵仔煎', '刈包', '手抓羊肉', '鳗鱼饭', '狗不理包子', '扣三丝']
食用频率为3的有131个
['龙眼', '海鲜砂锅粥', '蒜泥白肉', '猪肚鸡', '瑞士卷', '荔枝肉', '腊味煲仔饭', '涮羊肉', '老醋蛰头', '清蒸武昌鱼']
食用频率为4的有163个
['鸡蛋灌饼', '豆花饭', '水煮鱼', '蛤蜊蒸蛋', '葱爆羊肉', '芹菜炒粉条', '小米粥', '重庆小面', '口水鸡', '外婆菜炒鸡蛋']
食用频率为5的有62个
['干煸四季豆', '家常豆腐', '蒜蓉炒菜心', '牛腩面', '韭黄炒蛋', '手撕包菜', '鱼香肉丝饭', '酱油炒饭', '炒空心菜', '肉包']
**************************************************
熟悉程度为1的有3个
['蟠龙菜', '虎皮毛豆腐', '凉拌沙葱']
熟悉程度为2的有27个
['软兜长鱼', '搅团', '潮汕生腌', '手把肉', '蜜汁火方', '鸡汤氽海蚌', '拆烩鲢鱼头', '冬瓜盅', '问政山笋', '鼎湖上素']
熟悉程度为3的有79个
['汽锅鸡', '沙茶面', '馕', '懒龙', '冰糖湘莲', '藜蒿炒腊肉', '雪衣豆沙', '过油肉', '葱烧海参', '炒牛肝菌']
熟悉程度为4的有177个
['酒酿圆子', '熏鸭', '钵钵鸡', '黄焖羊肉', '牛肉汤', '油爆虾', '芋头

# 生成常见早餐的用例
用例数太少了所以只能把频率放到4

In [ ]:
res = df[(df['食用频率'] >= 4) & (df['熟悉程度'] == 5)  & (df['早中晚餐消费场景'] == '早餐')]
print(f'食用频率>=4 & 熟悉程度为=5 的 早餐 有{len(res)}个')
random_sample_of_res = res.sample(n=300, random_state=42)
print(random_sample_of_res.iloc[:, 0].to_list())
res_test = df_test[(df_test['食用频率'] >= 4) & (df_test['熟悉程度'] == 5)  & (df_test['早中晚餐消费场景'] == '早餐')]
print(f'test中 食用频率>=4 & 熟悉程度为=5 的 早餐 有{len(res_test)}个')
print(res_test.iloc[:, 0].to_list())
intersection_set = set(res_test.iloc[:, 0].to_list()).intersection(set(random_sample_of_res.iloc[:, 0].to_list()))
print(intersection_set)
random_sample_of_res = pd.concat([res_test, random_sample_of_res])
random_sample_of_res.drop_duplicates(subset=['菜品名'], inplace=True, keep='first')
# random_sample_of_res.sort_values(by='菜品名')
random_sample_of_res.insert(0, "sample_id", range(len(random_sample_of_res)))
random_sample_of_res.to_csv(os.path.join(csv_root, 'C1_testcase_breakfast_freq4_familarity5.csv'), index=False, encoding='gbk')

食用频率>=4 & 熟悉程度为=5 的 早餐 有583个
['芝麻鸡蛋饼', '五香萝卜干', '猪肉大葱包子', '香菇青菜包', '芝士吐司', '葱油鸡蛋饼', '白菜肉馅包子', '肉松', '豆角猪肉包子', '芒果酸奶', '蔬菜包', '三鲜包子', '培根煎蛋', '香酥油条', '土豆饼', '千层饼', '白水煮蛋', '白菜猪肉馅的包子', '麦包', '炸油条', '黄油吐司', '奶油小面包', '冰豆浆', '酱油荷包蛋', '五香鸡蛋', '香菇瘦肉粥', '菠菜鸡蛋粥', '馅饼', '熟鸡蛋', '牛奶燕麦粥', '煎饼果子', '大白馒头', '小笼包', '金牛角面包', '小包子', '脆麦片', '火腿三明治', '豆沙毛毛虫面包', '火龙果酸奶', '豆沙小馒头', '韭菜鸡蛋盒子', '奶油面包', '家庭油条', '五谷杂粮粥', '大葱肉包', '奶香面包', '豆浆油条', '红豆餐包', '腐乳馒头', '早餐三明治', '白菜猪肉包子', '芝麻煎饼', '包子', '小油条', '萝卜咸菜', '白吐司面包', '豆包', '早餐手抓饼', '肠仔包', '广式葱油饼', '卤肉包子', '水果汁', '煎蛋三明治', '猪肉粥', '包子馒头花卷', '发面葱油饼', '香肠包', '花卷馒头', '碧绿青菜包', '糙米粥', '东北煎饼', '葱香薄饼', '白糖烧饼', '大肉包子', '鸡蛋薄饼', '葱花鸡蛋饼', '青菜皮蛋粥', '发面包子', '全麦切片面包', '超绵吐司', '葱油花卷', '五谷粥', '杭州小笼包', '水波蛋', '红萝卜鸡蛋饼', '圆白菜包子', '葱香火腿面包', '草莓酱面包', '蜂蜜酸奶', '鸡肉三明治', '韭菜鸡蛋馅包子', '葱花油饼', '烤烧饼', '豆腐脑', '青菜香菇包', '葱香肉饼', '咖啡牛奶', '黄豆豆浆', '红糖全麦馒头', '葱花烙饼', '炒蛋', '鲜肉包', '杂粮包', '小笼包子', '芝麻饼', '香酥鸡蛋灌饼', '起酥面包', '鸡蛋火腿三明治', '青菜香菇包子', '老式油条', '摊煎饼', '白菜肉包子', '葱油酥饼', '巧克力牛奶', '葱油煎饼', '牛奶馒头', '发面红糖饼', '

# 生成常见午晚餐的用例

In [ ]:
res = df[(df['食用频率'] == 5) & (df['熟悉程度'] == 5)  & (df['早中晚餐消费场景'] == '午餐或晚餐')]
print(f'食用频率=5 & 熟悉程度为=5 的 午餐或晚餐 有{len(res)}个')
random_sample_of_res = res.sample(n=500, random_state=42)
print(random_sample_of_res.iloc[:, 0].to_list())
res_test = df_test[(df_test['食用频率'] == 5) & (df_test['熟悉程度'] == 5)  & (df_test['早中晚餐消费场景'] == '午餐或晚餐')]
print(f'test中 食用频率=5 & 熟悉程度为=5 的 午餐或晚餐 有{len(res_test)}个')
print(res_test.iloc[:, 0].to_list())
intersection_set = set(res_test.iloc[:, 0].to_list()).intersection(set(random_sample_of_res.iloc[:, 0].to_list()))
print(intersection_set)
random_sample_of_res = pd.concat([res_test, random_sample_of_res])
random_sample_of_res.drop_duplicates(subset=['菜品名'], inplace=True, keep='first')
# random_sample_of_res.sort_values(by='菜品名')
random_sample_of_res.insert(0, "sample_id", range(len(random_sample_of_res)))
random_sample_of_res.to_csv(os.path.join(csv_root, 'C2_testcase_lunch_and_dinner_freq5_familarity5.csv'), index=False, encoding='gbk')

食用频率=5 & 熟悉程度为=5 的 午餐或晚餐 有938个
['杂酱面', '清蒸草鱼', '清汤挂面', '葱炒蛋', '奥尔良鸡腿堡', '炖大白菜', '红烧排骨饭', '肉末鸡蛋羹', '葱花摊鸡蛋', '大蒜炒肉', '肉炒油菜', '蛋炒面', '菜豌豆', '猪肉胡萝卜饺子', '西红柿盖浇饭', '韭菜虾仁炒蛋', '猪肉芹菜水饺', '腌菜', '西兰花炒肉', '醋熘土豆丝', '蔬菜丁', '胡萝卜丝炒鸡蛋', '玉米鸡丁', '海米白菜汤', '紫菜虾皮汤', '西红柿鸡蛋汤', '芹菜豆干炒肉丝', '紫皮洋葱', '青椒炒土豆丝', '胡萝卜猪肉饺', '香菇鸡肉', '蒜黄炒肉', '清炒菠菜', '素炒豆角', '西红柿紫菜蛋汤', '爆炒卷心菜', '凉拌黄瓜丝', '卷心菜炒肉', '肉末茄子', '北方水饺', '清爽小菜', '青菜面条', '香辣土豆丝', '麻婆豆腐', '炒鸡肉片', '菠菜蘑菇鸡蛋汤', '香菇鸡腿', '拌面条', '素炒油麦菜', '菠菜炒肉丝', '手撕白菜', '不辣的宫保鸡丁', '猪肉玉米饺', '大蒜炒肉片', '鸡蛋韭菜虾仁饺子', '炒什蔬', '发面馒头', '小白菜豆腐汤', '烙饼', '煮好的米饭', '白菜炒蘑菇', '尖椒炒鸡蛋', '辣炒圆白菜', '油菜心', '土豆片', '素炒土豆丝', '炒波菜', '辣椒炒鸡蛋', '白菜炖肉', '青菜丸子汤', '炒白菜梗', '拌小菜', '湖南小炒肉', '菠菜鸡蛋汤', '火腿鸡蛋羹', '素炒卷心菜', '椰菜', '炒青椒肉丝', '牛肉面', '凉拌青瓜', '酱油蛋炒饭', '清炒青菜', '泡菜', '炒长豆角', '牛肉炒青椒', '清炒菜心', '辣椒炒肉丝', '清炒圆白菜', '炝炒莲白', '辣酱拌面', '清爽小炒', '宫保肉丁', '米粉', '炒牛肉', '广州菜心', '芹菜炒猪肉', '白菜粉丝汤', '包菜炒五花肉', '猪肉芹菜馅饺子', '三鲜炒面', '土豆丝炒肉丝', '小炒菜', '韭菜炒肉丝', '韭菜炒肉片', '土豆炒肉', '素炒青菜', '拌面', '炒虾仁', '青椒炒鸡块', '素菜饺子', '尖椒炒肉', '炒老豆腐', '蒜黄炒鸡蛋', '清炒莴笋', '

# 生成常见记忆菜的用例
记忆菜感觉不是很常吃，食用频率放到了3，熟悉程度放到了4

In [ ]:
res = df[(df['食用频率'] >= 3) & (df['熟悉程度'] >= 4)  & (df['记忆/推理菜'] == '记忆菜')]
print(f'食用频率>=3 & 熟悉程度为>=4 的 记忆菜 有{len(res)}个')
random_sample_of_res = res.sample(n=400, random_state=42)
print(random_sample_of_res.iloc[:, 0].to_list())
res_test = df_test[(df_test['食用频率'] >= 3) & (df_test['熟悉程度'] >= 4)  & (df_test['记忆/推理菜'] == '记忆菜')]
print(f'test中 食用频率>=3 & 熟悉程度为>=4 的 记忆菜 有{len(res_test)}个')
print(res_test.iloc[:, 0].to_list())
intersection_set = set(res_test.iloc[:, 0].to_list()).intersection(set(random_sample_of_res.iloc[:, 0].to_list()))
print(intersection_set)
random_sample_of_res = pd.concat([res_test, random_sample_of_res])
random_sample_of_res.drop_duplicates(subset=['菜品名'], inplace=True, keep='first')
# random_sample_of_res.sort_values(by='菜品名')
random_sample_of_res.insert(0, "sample_id", range(len(random_sample_of_res)))
random_sample_of_res.to_csv(os.path.join(csv_root, 'C3_testcase_memory_freq3_familarity4.csv'), index=False, encoding='gbk')

食用频率>=3 & 熟悉程度为>=4 的 记忆菜 有1342个
['法兰西多士', '鲜虾肠粉', '三鲜日本豆腐', '水煮牛肉', '奥良烤鸡翅', '红油钵钵鸡', '馋嘴牛蛙', '鱼蛋', '粿条', '客家梅菜扣肉', '豆沙绿', '麻薯球', '格兰诺拉麦片', '当红辣子鸡', '香炒花甲', '美式炒蛋', '凉拌千张丝', '菠萝油', '避风塘炒虾', '炒和菜', '白汤肥肠粉', '炒蛋多士', '日式寿司', '培根肉', '醪糟蛋', '多芒小丸子', '春卷', '三黄鸡汤', '麻辣串串香', '乌江榨菜', '酱炒三丁', '酱汤', '虾仁跑蛋', '鸭血粉丝', '紫菜手卷', '凤尾西芹', '东北凉菜', '干炒牛河', '泰式冬阴功汤', '肉松面包', '家庭油条', '养乐多', '奥尔良牛肉条', '宫爆杏鲍菇', '毛家红烧肉', '口水鸡腿', '豆沙毛毛虫面包', '水煮鱼肉片', '酸菜', '大舅舅白斩鸡', '南昌拌粉', '广东老火汤', '海蛎煎', '炖茄盒', '寿司', '葱香豉油鸡', '日式照烧鸡排饭', '愤怒的汉堡', '腐乳红烧肉', '五香鸡爪', '素三鲜饺子', '白玉苦瓜', '木须柿子', '菠萝包', '韩式酱汤', '溏心煎蛋', '猪油炒双花', '照烧鸡', '豆花', '白萝卜红烧肉', '吐司面包片', '酱肘花', '土豆疙瘩汤', '沙朗牛排', '奥尔良果香烤鸡', '腊肠菜', '水煮鱼片', '宫保鱼丁', '玛格丽特比萨', '金沙玉米', '猪肉烧卖', '广东靓汤', '海鲜拉面', '奥尔良炸鸡排', '重庆水煮鱼', '奥尔良鸡肉披萨', '盐菜扣肉', '扒牛肉', '啤酒烧鱼块', '尊享布朗尼蛋糕', '韩国部队汤', '东北酱肘子', '豪华坚果沙琪玛', '拌榨菜丝', '雪菜肉丝', '海苔肉松小贝', '柱候酱牛肉', '糯米烧卖', '芙蓉蔬菜汤', '朝鲜族辣白菜', '焦糖玛奇朵', '冰咖啡雪顶', '炝拌小菜', '周黑鸭', '部队汤', '避风塘虾', '虾面', '红枣枸杞银耳羹', '菠萝菠萝包', '总会三明治', '白切牛肉', '红烧羊蝎子', '冰花水煎包', '水煎包子', '冰糖红烧肉', '红油

# 生成常见推理菜用例

In [ ]:
res = df[(df['食用频率'] == 5) & (df['熟悉程度'] == 5)  & (df['记忆/推理菜'] == '推理菜')]
print(f'食用频率=5 & 熟悉程度为=5 的 推理菜 有{len(res)}个')
random_sample_of_res = res.sample(n=500, random_state=42)
print(random_sample_of_res.iloc[:, 0].to_list())
res_test = df_test[(df_test['食用频率'] == 5) & (df_test['熟悉程度'] == 5)  & (df_test['记忆/推理菜'] == '推理菜')]
print(f'test中 食用频率==5 & 熟悉程度为==5 的 推理菜 有{len(res_test)}个')
print(res_test.iloc[:, 0].to_list())
intersection_set = set(res_test.iloc[:, 0].to_list()).intersection(set(random_sample_of_res.iloc[:, 0].to_list()))
print(intersection_set)
random_sample_of_res = pd.concat([res_test, random_sample_of_res])
random_sample_of_res.drop_duplicates(subset=['菜品名'], inplace=True, keep='first')
# random_sample_of_res.sort_values(by='菜品名')
random_sample_of_res.insert(0, "sample_id", range(len(random_sample_of_res)))
random_sample_of_res.to_csv(os.path.join(csv_root, 'C4_testcase_infer_freq5_familarity5.csv'), index=False, encoding='gbk')

食用频率=5 & 熟悉程度为=5 的 推理菜 有1034个
['尖椒鸡蛋', '清炒小白菜', '宝宝小馒头', '黄焖鸡米饭', '酸辣圆白菜', '香菇鸡肉', '耗油青菜', '披萨饼', '猪肉鲜蘑馅饺子', '水饺', '豆芽炒肉丝', '素炒卷心菜', '冰绿茶', '爆炒鸡', '豆腐白菜', '白菜肉馅包子', '白菜肉水饺', '肉丁炒饭', '奥尔良蜂蜜烤翅', '酱烧茄子', '炒素菜', '自发粉馒头', '土豆丝炒肉', '糖醋带鱼', '土豆片', '青笋木耳肉片', '凉拌小黄瓜', '素炒豆角', '手撕包心菜', '炒鸡肉丝', '白菜鸡蛋饺子', '手工水饺', '红烧鸡腿', '手工饺子', '炸薯条', '韭菜猪肉饺子', '蒜泥青菜', '香菇鸡蛋汤', '木耳炒鸡蛋', '白面馒头', '青椒炒肉', '肉末炖蛋', '蛋花汤', '炒蛋', '包子馒头花卷', '排骨萝卜汤', '清炒莴笋丝', '辣椒炒鸡蛋', '标准面包', '葱油发面饼', '炒圆白菜', '西红柿炒鸡蛋面', '拌黄瓜丝', '菜汤面', '素菜馅饺子', '全麦馒头', '大拌菜', '腌萝卜', '炝油麦菜', '大葱肉包', '水饺汤', '小白菜炖豆腐', '凉拌黄瓜条', '猪肉白菜饺', '煎蛋三明治', '凉拌波菜', '白菜蛋炒饭', '滑蛋虾仁', '醋熘白菜', '西红柿蛋花汤', '香菇菜包', '盒饭', '热牛奶', '煎鸡翅', '炒饭', '冬瓜排骨汤', '辣炒白菜', '鸡蛋火腿肠炒饭', '白菜肉包子', '营养排骨汤', '肉烧豆腐', '全麦切片面包', '猪肉韭菜包子', '酸辣手撕包菜', '土豆炒肉丝', '爆炒白菜', '大米粥', '牛奶麦片', '炒虾仁', '酱油挂面', '青菜炒蘑菇', '猪肉白菜水饺', '肉炒土豆丝', '鸡蛋炒油菜', '彩椒蛋炒饭', '韭菜炒肉', '西红柿青菜蛋汤', '菠菜炖豆腐', '简易面包', '鸡蛋豆腐汤', '韭菜猪肉馅饺子', '炒虾', '火腿鸡蛋炒饭', '酸辣白菜', '油菜炒肉', '口蘑炒鸡蛋', '菜肉包子', '香菇烧豆腐', '拌木耳', '凉拌黄瓜木耳', '炒什蔬', '炒白菜片', '白菜粉条炖肉', '西兰花炒蛋', '米粉

# 生成常见主食用例
主食把食用频率放到了4

In [ ]:
res = df[(df['食用频率'] >= 4) & (df['熟悉程度'] == 5) & (df['营养建议分类'].str.contains('主食'))]
print(f'食用频率>=4 & 熟悉程度为=5 的 主食 有{len(res)}个')
random_sample_of_res = res.sample(n=400, random_state=42)
print(random_sample_of_res.iloc[:, 0].to_list())
res_test = df_test[(df_test['食用频率'] >= 4) & (df_test['熟悉程度'] == 5)  & (df_test['营养建议分类'].str.contains('主食'))]
print(f'test中 食用频率>=4 & 熟悉程度为==5 的 主食 有{len(res_test)}个')
print(res_test.iloc[:, 0].to_list())
intersection_set = set(res_test.iloc[:, 0].to_list()).intersection(set(random_sample_of_res.iloc[:, 0].to_list()))
print(intersection_set)
random_sample_of_res = pd.concat([res_test, random_sample_of_res])
random_sample_of_res.drop_duplicates(subset=['菜品名'], inplace=True, keep='first')
# random_sample_of_res.sort_values(by='菜品名')
random_sample_of_res.insert(0, "sample_id", range(len(random_sample_of_res)))
random_sample_of_res.to_csv(os.path.join(csv_root, 'C5_testcase_main_food_freq4_familarity5.csv'), index=False, encoding='gbk')

食用频率>=4 & 熟悉程度为=5 的 主食 有1272个
['标准面包', '大白馒头', '猪肉韭菜馅水饺', '海鲜馄饨', '西红柿拌面', '馅饼', '黑米粥', '煎吐司', '猪肉香葱水煎包', '果酱三明治', '至尊披萨', '鸡蛋灌饼', '青菜面', '西红柿鸡蛋挂面汤', '牛奶燕麦粥', '肉丝鸡蛋面', '牛奶蛋饼', '虾皮面', '糙米饭', '牛腩面', '蛋煎馒头片', '韭菜猪肉饺子', '酸菜蒸饺', '混沌皮', '韭菜鸡蛋饼', '猪肉豆芽菜煎饺', '红豆黑米粥', '羊肉蒸饺', '大肉包', '早餐手抓饼', '西葫芦鸡蛋饺子', '猪肉芹菜水饺', '猪肉馅饺子', '猪肉韭菜饺子', '咖喱猪肉饭', '三丝炒面', '粗粮馒头', '煎饼', '羊角面包', '紫菜虾皮面', '自发粉馒头', '水面', '牛肉面', '火腿面包', '大肉包子', '风味凉面', '白花卷', '奶香吐司面包', '夏威夷披萨', '香菇青菜面', '辫子豆沙包', '小笼包', '紫菜鸡蛋面', '茄子打卤面', '杭州小笼包', '蛋炒面', '煎蛋面', '小白菜粉丝汤', '意面肉酱', '圆白菜馅饺子', '营养八宝粥', '素饺', '兰州牛肉', '纯米粉', '青菜粉丝', '辣酱拌面', '葱花鸡蛋煎饼', '韭菜馅饺子', '白面烙饼', '脆麦片', '葱油酥饼', '老坛酸菜牛肉面', '水煮玉米', '香菇瘦肉粥', '白白吐司', '炒米', '茄汁意面', '手工面条', '紫菜面', '菜肉粥', '白菜猪肉包', '番茄煎蛋面', '兰州牛肉拉面', '炒馒头丁', '番茄牛腩面', '鸡肉炒饭', '榨菜肉丝饭', '鸡蛋盖饭', '发面糖饼', '鸡肉炖粉条', '发面葱油酥饼', '白菜粉条素包子', '葱油汤面', '水饺', '青菜面条', '辛拉面', '番茄肉酱面', '全麦面包片', '红糖包', '白菜猪肉馅饺子', '煮米线', '南瓜花卷', '山东大馒头', '白菜香菇猪肉饺子', '咖喱牛肉饭', '韭菜虾仁饺子', '缤纷炒饭', '辣酱面', '绿豆小米粥', '萝卜馅包子', '鸡蛋番茄面', '瘦肉粥', '皮蛋粥', '牛奶麦片', '腊肠炒饭', '香

# 生成主食材和烹饪方式的用例

In [ ]:

i = 6
df_source = pd.concat([df, df_test])
df_source = df_source.drop_duplicates(subset=["菜品名"])
df_source = df_source.reset_index(drop=True)
for key_to_be_choose in ["ingrdient_freq", "cooking_method"]:
    classification_requirement_csv_now = (
        r"/date0/crwu/zjts_foodname/data/classification/data/classification_requirement_"
        + key_to_be_choose
        + ".csv"
    )
    df_classification_now = read_dataset(classification_requirement_csv_now)
    df_classification_now = df_classification_now.sort_values(
        "最低占比", ascending=False
    )
    print(df_classification_now)
    print(sum(df_classification_now["最低占比"]))
    df_classification_now.to_csv(classification_requirement_csv_now, index=False, encoding='gbk')

    sample_num = 500
    conditions = {}
    conditions = defaultdict(dict)
    for row in df_classification_now.iterrows():
        temp_dict = row[1].to_dict()
        # print(temp_dict.get('出现频率调整', 0))
        # print(temp_dict.get('是否常见', 0))
        target_num = int(
            (temp_dict["最低占比"] * (1 + temp_dict.get("出现频率调整", 0) * 0.33))
            * temp_dict.get("是否常见", 1)
            * sample_num
        )
        print(temp_dict["标签内容"], target_num)
        conditions[temp_dict["分类标签"]][temp_dict["标签内容"]] = target_num
    print(conditions)

    print("\n使用迭代式条件满足抽样:")
    iter_times = 0
    max_iter_times = 10
    all_conditions_met = False
    while all_conditions_met is False and iter_times < max_iter_times:
        iter_times += 1
        print(f"\n第{iter_times}次尝试抽取{sample_num}个样本:")
        sampled_df, all_conditions_met = iterative_conditional_sampling_fuzzy_optimized(
            df_source, sample_num, conditions, random_state=42
        )
    sampled_df.sort_values(by=["菜品名"], inplace=True)
    sampled_df.insert(0, "sample_id", range(len(sampled_df)))
    target_path = os.path.join(csv_root, f"C{i}_testcase_{key_to_be_choose}.csv")
    sampled_df.to_csv(target_path, index=False, encoding='gbk')
    print(f"output to {target_path}")
    i += 1

2025-10-14 23:03:46 - file_deal - INFO - [file_deal.py:151] - Loading dataset from CSV: /date0/crwu/zjts_foodname/data/classification/data/classification_requirement_ingrdient_freq.csv ...


      分类标签      标签内容   最低占比  是否常见  出现频率调整      备注
0   主要食材种类    面粉或小麦类  0.211     1       0     NaN
1   主要食材种类        猪肉  0.188     1       0     NaN
2   主要食材种类       叶菜类  0.157     1      -2     NaN
3   主要食材种类       根茎类  0.151     1      -2     NaN
4   主要食材种类        米类  0.111     1       0     NaN
5   主要食材种类        鸡蛋  0.110     1       0     NaN
6   主要食材种类     奶或奶油类  0.071     1       0     NaN
7   主要食材种类         鱼  0.066     1       0     NaN
8   主要食材种类        鸡肉  0.064     1       0     NaN
9   主要食材种类     大豆或豆类  0.058     1       0     NaN
10  主要食材种类       瓜果类  0.056     1      -2     NaN
11  主要食材种类       菌菇类  0.056     1       0     NaN
12  主要食材种类    坚果或种子类  0.042     1       0     NaN
13  主要食材种类        虾类  0.041     1       0     NaN
14  主要食材种类    豆腐或豆干类  0.040     1       0     NaN
15  主要食材种类       茄果类  0.039     1      -2     NaN
16  主要食材种类  热带或亚热带水果  0.035     0      -2     NaN
17  主要食材种类        牛肉  0.033     1       1     NaN
18  主要食材种类    仁果或核果类  0.029     0       0     NaN
